In [ ]:
import random

class Room_Effect: #room effects that occur upon entering
    def __init__(self, name, description):
        self.name = name
        self.description = description
    
    def activate_effect(self, character):
        print(f"Upon entering the room, you find: {self.name} - {self.description}")
        if self.name == "the room on fire": #a way to force player to move more
            print(f"{character.name} suffers one light wound if they remain here")
        elif self.name == "the room under lockdown": #prevent player from going back to previous room
            print(f"{character.name} sees the door lock behind them")
        elif self.name == "a specimen staring back at you": #start encounter
            print(f"{character.name} is now in combat with") #add {enemy.name} later when they are made
        elif self.name == "a place to get your bearings": #neutral room
            print("there is nothing to worry about in this room.")

Room_Effect_Pool = [ #still need to code in effects
    Room_Effect("the room on fire", "if in room at end of turn, suffer one light wound"),
    Room_Effect("the room under lockdown", "unable to move to previous room"),
    Room_Effect("a specimen staring back at you", "starts an encounter"),
    Room_Effect("a place to get your bearings", "nothing happens")
]

#create rooms and keep track of whether they've been visited
class Room:
    def __init__(self, room_id):
        self.room_id = room_id
        self.visited = False
        self.effect = None
    
    def visit(self, character):
        if not self.visited:
            self.visited = True
            self.apply_random_effect(character)
    
    def apply_random_effect(self, character):
        self.effect = random.choice(Room_Effect_Pool)
        self.effect.activate_effect(character)

class Character_Inventory: #managable inventory system to add/remove gained items
    def __init__(self):
        self.items = [] 
    
    def add_item(self, item):
        self.items.append(item)
        print(f"You find a(n) {item} and add it to your bag.")
    
    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)
            print(f"You drop the {item}.")
        else:
            print(f"{item} is not in your inventory.")
    
    def view_inventory(self): #need to add way for user to add/remove items. Can be done as free action
        if self.items:
            print("You peer into your bag, finding: ")
            for item in self.items:
                print(f" - {item}")
        else:
            print("The bag you are holding appears to be empty. You should take some time to fill it.")

class Character_Class: #initialize character classes/weapons/challenges for extra items
    def __init__(self, name, weapon, challenges):
        self.name = name
        self.weapon = weapon
        self.inventory = Character_Inventory() #inventory

    def complete_challenge(self, challenge_name): #saving the rest of this for later
        if challenge_name in self.challenges and not self.challenges[challenge_name]['completed']:
            self.challenges[challenge_name]['completed'] = True
            reward_item = self.challenges[challenge_name]['reward']
            self.unlock_item(reward_item)
            print(f"Task '{challenge_name}' complete. The company grants you: {reward_item}.")
            print(f"You pocket the {reward_item}.")
        else:
            print(f"Task '{challenge_name}' already complete or does not exist.")

class Character_Creator:
    def create_character(class_name, name):
        class_map = {
            'Engineer' : ('Plasma Cutter', Character_Class, {
                'Upload Data' : {'completed': False, 'reward': 'Computer'}, #Scan bodies? Computer can be used in certain rooms to activate their ability
                'Repair Systems' : {'completed': False, 'reward': 'Toolkit'} #Repair like power or something. Toolkit for crafting? or easy repair of something?
            }),
            'Overseer' : ('Pistol', Character_Class, {
                'Clearance Check' : {'completed': False, 'reward': 'Access Pass'}, #Not sure how to implement this yet. Access Pass would open locked doors
                'Camera Check' : {'completed': False, 'reward': 'Data Pad'} #Check cameras. Data Pad could check cameras for info at any time
            }),
            'Scientist' : ('Experimental Rifle', Character_Class, {
                'Study Specimens' : {'completed': False, 'reward': 'Specimen Research Sheet'}, #scan a specimen in scan room. grants knowledge of a weakness
                'Craft with Flames' : {'completed': False, 'reward': 'Flamethrower'} #craft flamethrower, does random damage based on 2 d6 and makes enemy retreat
            }),
            'Security Guard' : ('Assault Rifle', Character_Class, {
                'Breach the Area' : {'completed': False, 'reward': 'Taser'}, #Explore 4 rooms. Taser allows escape from room without triggering encounter with 4? uses
                'Clear the Area' : {'completed': False, 'reward': 'Exoskeleton'} #Kill 2? enemies. Exoskeleton prevents x damage (need to design monsters still)
            })
        }

        if class_name in class_map:
            weapon, character_class, challenges = class_map[class_name]
            return character_class(name, weapon, challenges)
        else:
            print ("The company does not employ that profession.")
            return None

def explore_room(character): #room exploration
    rooms = [Room(i) for i in range(1, 16)] #set of 15 rooms
    
    random.shuffle(rooms) #randomize rooms

    current_room_index = 0 #start from first room

    while True: #movement through rooms
        print(f"\nYou are currently in Room {rooms[current_room_index].room_id}.")
        print("Choose your direction: ")
        print("1. Move Forward")
        print("2. Move Backward")
        print("3. Stay Put")

        choice = input("How would you like to move? (1/2/3): ").strip()
        
        if choice == '3':
            print("You decide to stay put.")
            break
        elif choice == '1':
            if current_room_index < len(rooms) - 1:
                current_room_index += 1
                print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
            else:
                print("You are at the last room. You can only go backward")
        elif choice == '2':
            if current_room_index > 0:
                current_room_index -= 1
                print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
            else:
                print("You are currently in the first room. You can only go forward")
        else:
            print("Invalid choice. Please enter 1 (Forward), 2 (Backward), or 3 (Stay Put)")

        rooms[current_room_index].visit(character)

def class_pick():
    while True: #run loop until class is chosen or user chooses to break after selecting wrong
        print("Choose your character: ")
        print("1. Engineer")
        print("2. Overseer") 
        print("3. Scientist") 
        print("4. Security Guard")  

        user_choice = input("Please pick the number corresponding to your desired class: ")

        class_choices = {
            '1': 'Engineer',
            '2' : 'Overseer',
            '3' : 'Scientist',
            '4' : 'Security Guard' 
        }

        if user_choice in class_choices:
            class_name = class_choices[user_choice]
            name = input(f"What's your name?: ")
            character = Character_Creator.create_character(class_name, name)
            final_choice = input(f"So you are {character.name} the {class_name}?").lower()

            if final_choice == 'y' or final_choice == 'yes':
                print(f"Welcome {character.name} the {class_name}! \nPlease take the provided {character.weapon}.") 
                print("The company has been authorized to offer additional support should you seek it, but they require additional tasks to be completed in return.")
                print("The tasks are as follows: ")
                
                explore_room(character) #start exploration post character creation. need to add action system and other actions
 
                break
        else: 
            print("\nI'm sorry, you don't seem to be in our database.")
            try_again = input("Would you like to try again?: ").lower()

            if try_again != 'y' and try_again != 'yes':
                print ("Server Locking")
                break
class_pick()